# 第3章 字典和集合

## 3.1 字典的现代句法

1. 字典推导式

In [3]:
dial_codes = [
    (880, 'Bangladesh'),
    (55, 'Brazil'),
    (86, 'China'),
    (91, 'India'),
    (62, 'Indonesia'),
    (81, 'Japan'),
    (234, 'Nigeria'),
    (92, 'Pakistan'),
    (7, 'Russia'),
    (1, 'United States'),
]
# 对调键和值
country_dial = {country: code for code, country in dial_codes}
country_dial

{'Bangladesh': 880,
 'Brazil': 55,
 'China': 86,
 'India': 91,
 'Indonesia': 62,
 'Japan': 81,
 'Nigeria': 234,
 'Pakistan': 92,
 'Russia': 7,
 'United States': 1}

In [4]:
# 按国家名称排序，再次对调键和值，把值转成大写，筛选code<70的项
{code: country.upper()
    for country, code in sorted(country_dial.items())
    if code < 70}

{55: 'BRAZIL', 62: 'INDONESIA', 7: 'RUSSIA', 1: 'UNITED STATES'}

2. 映射拆包

In [5]:
def dump(**kwargs):
    return kwargs

dump(**{'x' :1}, y=2, **{'z': 3})

{'x': 1, 'y': 2, 'z': 3}

In [6]:
{'a': 0, **{'x': 1}, 'y': 2, **{'z': 3, 'x': 4}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3}

3. 使用`|`合并映射

In [7]:
d1 = {'a': 1, 'b': 3}
d2 = {'a': 2, 'b': 4, 'c': 6}
d1 | d2

{'a': 2, 'b': 4, 'c': 6}

## 3.2 使用模式匹配处理映射

In [1]:
# 从出版物记录中提取创作者的名字
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author': name}:
            return [name]
        case {'type': 'book'}:
            return ValueError(f"Invalid 'book' record: {record!r}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _:
            return ValueError(f'Invalid record: {record!r}')

In [2]:
b1 = dict(api=1, author='Douglas Hofstadter', type='book', title='Godel, Escher, Bach')
get_creators(b1)

['Douglas Hofstadter']

In [3]:
from collections import OrderedDict

b2 = OrderedDict(api=2, type='book', title='Python in a Nutshell', authors='Martelli Ravenscroft Holden'.split())
get_creators(b2)

['Martelli', 'Ravenscroft', 'Holden']

In [4]:
get_creators({'type': 'book', 'pages': 770})

ValueError("Invalid 'book' record: {'type': 'book', 'pages': 770}")

In [5]:
get_creators('Spam, spam, spam')

ValueError("Invalid record: 'Spam, spam, spam'")

## 3.3 自动处理丢失的键

In [19]:
# 在查找键时把非字符串键转换成字符串
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [20]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [21]:
d[4]

'four'

In [23]:
d.get('2')

'two'

In [24]:
d.get(4)

'four'

In [25]:
d.get(1, 'N/A')

'N/A'

## 3.4 dict的变体

1.collections.OrderedDict

- 等值检查考虑顺序
- 方便执行重新排序操作，空间利用率、迭代速度和更新操作的性能是次要的。
- 从算法上看，OrderedDict处理频繁重新排序操作的效果比dict好，适合用于跟踪近期存取情况。

2.collections.ChainMap

- 存放一组映射，可作为一个整体来搜索。
- 查找操作按照输入映射在构造函数调用中出现的顺序执行，一旦找到指定的键，立即结束。
- 不复制输入映射，存放映射的引用。
- 更新或插入操作只影响第一个输入映射。

3.collections.Counter

- 一种对键计数的映射，更新现有的键，计数随之增加。
- 可用于统计可哈希对象的实例数量。

4.shelve.Shelf

- 持久存储字符串键与Python对象之间的映射。

In [26]:
# StrKeyDict在插入、更新和查找时，始终把非字符串键转换为str类型
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

## 3.5 不可变映射

In [27]:
#从MappingProxyType根据dict对象构建只读的mappingproxy实例
from types import MappingProxyType

d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [28]:
d_proxy[1]

'A'

In [32]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [33]:
d_proxy[2]

'B'

## 3.6 集合论

集合的基本作用是去除重复项。

In [34]:
l = ['spam', 'spam', 'eggs', 'spam', 'bacon', 'eggs']
set(l)

{'bacon', 'eggs', 'spam'}

In [35]:
list(set(l))

['bacon', 'eggs', 'spam']

In [36]:
# 如果想要去除重复项，同时保留每一项首次出现位置的顺序
list(dict.fromkeys(l).keys())

['spam', 'eggs', 'bacon']

In [38]:
# 集合推导式
from unicodedata import name

print({chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')})

{'×', '%', '>', '¥', '§', '¬', '=', '°', '¶', '©', '¢', 'µ', '±', '#', '¤', '£', '$', '÷', '®', '<', '+'}
